<a href="https://colab.research.google.com/github/jonesmo/data_science_projects/blob/main/Communities_and_Crime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Communities and Violent Crime**


The goal of this project is to create a supervised learning model to predict the violent crime rate in a community based on 122 different attributes of the community.  The data set combines information from the 1990 US Census, law enforcement data from the 1990 [US LEMAS survey](https://www.bjs.gov/index.cfm?ty=dcdetail&iid=248), and crime data from the 1995 [FBI UCR](https://www.fbi.gov/services/cjis/ucr).  This data is 25-30 years old.

[Original Data](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime+Unnormalized)

My goal is to find out for myself which attributes of a community and its policing most correlate to violent crime.

##**------------Exploratory Data Analysis-------------------------**

###Import Libraries and Create Dataframe

In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.svm import SVR, SVC
import scipy.stats as stats
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats import boxcox
from scipy.stats import yeojohnson
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.dummy import DummyRegressor

from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn import tree
from IPython.display import Image
!pip install pydotplus
!brew install graphviz
import pydotplus
import graphviz
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn import linear_model
from scipy.stats import bartlett, levene
from statsmodels.tsa.stattools import acf
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
import matplotlib.patches as mpatches

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


/bin/bash: brew: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

raw_data = pd.read_csv('/content/gdrive/My Drive/Thinkful/Supervised Learning/Capstone_Supervised_Learning/crime_communities_data.txt', header=None)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


###Examine Raw Data

In [ ]:
raw_data

In [ ]:
raw_data.shape

The raw data has 2,215 instances and 147 attributes, including the target and identifying variables.  The dataset is relatively small and therefore likely to be noisy.

I will manually import the column names.

In [ ]:
raw_data.columns = ['community_name', 'state', 'county', 'community', 'fold', 'population', 'household_size', 'race_pct_black', 'race_pct_white', 'race_pct_asian', 
                    'race_pct_hispanic', 'age_pct_12to21', 'age_pct_12to29', 'age_pct_16to24', 'age_pct_65up', 'num_urban', 'pct_urban', 'median_hh_income', 'pct_w_wages', 'pct_w_self_emp', 
                    'pct_w_invest_inc', 'pct_w_socsec_inc', 'pct_w_public_asst', 'pct_w_retirement_inc', 'median_fam_income', 'per_capita_income', 'white_pci', 'black_pci', 'native_pci', 
                    'asian_pci', 'other_pci', 'hispanic_pci', 'num_in_poverty', 'pct_in_poverty', 'education_pct_under9', 'education_pct_not_HS', 'education_pct_collegeup', 
                    'pct_unemployed', 'pct_employed', 'pct_manufacturing', 'pct_prof_services', 'pct_manufacturing_2', 'pct_management', 'pct_male_divorced', 'pct_male_unmarried', 
                    'pct_female_divorced', 'pct_pop_divorced', 'ppl_per_family', 'pct_two_parent_hh', 'pct_kids_two_parent_hh', 'pct_kids_under4_twophh', 'pct_kids_12_17_twophh', 
                    'pct_moms_under6_work', 'pct_moms_under18_work', 'num_born_to_unmarried', 'pct_born_to_unmarried', 'num_immigrants', 'pctimm_3year_immigrants', 'pctimm_5year_immigrants', 
                    'pctimm_8year_immigrants', 'pctimm_10year_immigrants', 'pct_3year_immigrants', 'pct_5year_immigrants', 'pct_8year_immigrants', 'pct_10year_immigrants', 'pct_English_only', 
                    'pct_English_notwell', 'pct_large_family', 'pct_large_hh', 'ppl_per_hh', 'ppl_per_owner_occupied_hh', 'ppl_per_rental_hh', 'pct_ppl_owner_occupied', 'pct_ppl_dense_hh', 
                    'pct_units_under3bed', 'median_bedrooms', 'num_vacant', 'pct_housing_occupied', 'pct_hh_owner_occupied', 'pct_vacant_boarded_up', 'pct_vacant_over6months', 
                    'median_year_built', 'pct_hh_no_phone', 'pct_hh_no_plumbing', 'owner_occupied_lower_quartile', 'owner_occupied_median_value', 'owner_occupied_upper_quartile', 'owner_occupied_Qrange', 
                    'lower_quartile_rent', 'median_rent', 'upper_quartile_rent', 'rent_Qrange', 'median_rent_utilities', 'rent_as_pct_hh_income', 'mortgage_as_pct_hh_income', 'non_mortgage_owner_as_pct_hh_income', 
                    'num_homeless_shelters', 'num_homeless_street', 'pct_foreign_born', 'pct_from_same_state', 'pct_5year_home_residents', 'pct_5year_city_residents', 'pct_5year_state_residents', 
                    'num_full_time_police', 'full_time_police_per_100K', 'num_field_police', 'field_police_per_100K', 'requests_for_police', 'requests_for_police_per_100K', 'requests_for_police_per_officer', 
                    'police_per_100K', 'racial_match_police_community', 'pct_police_white', 'pct_police_black', 'pct_police_hispanic', 'pct_police_asian', 'pct_police_minority', 
                    'num_drug_officers', 'kinds_of_drugs_seized', 'police_overtime_worked', 'land_area', 'population_density', 'pct_public_transit', 'num_police_cars', 'police_budget', 
                    'pct_police_on_patrol', 'gang_unit_deployed', 'pct_police_drug_unit', 'police_budget_per_100K', 'num_murders', 'murders_per_100K', 'num_rapes', 'rapes_per_100K', 
                    'num_robberies', 'robberies_per_100K', 'num_assaults', 'assaults_per_100K', 'num_burglaries', 'burglaries_per_100K', 'num_larcenies', 'larcenies_per_100K', 
                    'num_auto_thefts', 'auto_thefts_per_100K', 'num_arsons', 'arsons_per_100K', 'violent_crimes_per_100K', 'nonviolent_crimes_per_100K']

###Nulls and Missing Values

In [ ]:
pd.set_option('display.max_rows', None)

raw_data.isnull().sum()*100/raw_data.isnull().count()

In [ ]:
pd.set_option('display.max_rows', 20)

The nulls in the data are represented by '?', so I will replace this with NaN to see how many nulls are present.

In [ ]:
raw_data = raw_data.replace('?', np.nan)

In [ ]:
pd.set_option('display.max_rows', None)

raw_data.isnull().sum()*100/raw_data.isnull().count()

In [ ]:
pd.set_option('display.max_rows', 20)

Columns 0 through 4 are identifying information for each location, which will be removed.

The final 18 columns are not predictive but are measures of different types of violent crime.  These will all be removed except the target, Violent Crimes Per Pop 100K.

I will also remove the columns with ~84% nulls.  These are all missing data from police departments.  Because of the current conversations around policing, I was hoping to include this data, but police department transparency is notoriously lacking in many cities.  For example, [here](https://invisible.institute/police-data) is a brief informative video on police data in Chicago, and [here](https://beta.cpdp.co/) is the organization's dashboard, after seven years of legal battles to obtain the data.

In [ ]:
cc_df = raw_data.drop(columns=['community_name', 'state', 'county', 'community', 'fold', 'num_full_time_police', 'full_time_police_per_100K', 'num_field_police', 
                               'field_police_per_100K', 'requests_for_police', 'requests_for_police_per_100K', 'requests_for_police_per_officer', 'police_per_100K', 
                               'racial_match_police_community', 'pct_police_white', 'pct_police_black', 'pct_police_hispanic', 'pct_police_asian', 'pct_police_minority', 
                               'num_drug_officers', 'kinds_of_drugs_seized', 'police_overtime_worked', 'num_police_cars', 'police_budget', 'pct_police_on_patrol', 
                               'gang_unit_deployed', 'police_budget_per_100K', 'num_murders', 'murders_per_100K', 'num_rapes', 'rapes_per_100K', 'num_robberies', 
                               'robberies_per_100K', 'num_assaults', 'assaults_per_100K', 'num_burglaries', 'burglaries_per_100K', 'num_larcenies', 'larcenies_per_100K', 
                               'num_auto_thefts', 'auto_thefts_per_100K', 'num_arsons', 'arsons_per_100K', 'nonviolent_crimes_per_100K'], axis=1)

cc_df = cc_df.dropna()

cc_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 2214
Data columns (total 103 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   population                           int64  
 1   household_size                       float64
 2   race_pct_black                       float64
 3   race_pct_white                       float64
 4   race_pct_asian                       float64
 5   race_pct_hispanic                    float64
 6   age_pct_12to21                       float64
 7   age_pct_12to29                       float64
 8   age_pct_16to24                       float64
 9   age_pct_65up                         float64
 10  num_urban                            int64  
 11  pct_urban                            float64
 12  median_hh_income                     int64  
 13  pct_w_wages                          float64
 14  pct_w_self_emp                       float64
 15  pct_w_invest_inc                     

I'll inspect the two columns with datatype object to see what's going on.

In [ ]:
cc_df['other_pci']

In [ ]:
cc_df['violent_crimes_per_100K']

These are stored as strings, so I'll convert them to floats.

In [ ]:
cc_df['other_pci'] = cc_df['other_pci'].astype(float)
cc_df['violent_crimes_per_100K'] = cc_df['violent_crimes_per_100K'].astype(float)

###Collinearity of Variables

Looking at these variables, it seems likely that some are highly collinear with each other.  With 102 variables, I'll use multiple steps to narrow them down.  First, I'll use large correlation matrices and scan visually to eliminate the most obvious offenders.  Then, I'll create a function to determine which pairs of variables are highly correlated and eliminate one of each pair.  As a threshold, I'm considering any correlation above 0.6 to be highly collinear, and where there are multiple highly collinear variables, I'll keep the one most correlated with crime rates.

In [ ]:
column_list = cc_df.columns

corr_matrix = cc_df[column_list].corr()

plt.figure(figsize=(150, 150))
sns.heatmap(corr_matrix, square=True, annot=True, linewidths=0.5, cmap="YlGnBu")
plt.show()

To start, I'll eliminate 18 variables.  Measures of homeless populations and number of vacant homes are highly correlated to poverty with little correlation to crime rate. Measures of specific housing costs for owner-occupied and rental households are highly correlated to per capita income and to each other. Statistics about family size and household size are highly correlated with overall household size. The numbers of children born to unmarried parents and the number of immigrants are highly correlated to both their percentages and to urbanness.

In [ ]:
cc_df = cc_df.drop(columns=['num_homeless_shelters', 'num_homeless_street', 'owner_occupied_lower_quartile', 'owner_occupied_median_value', 'owner_occupied_upper_quartile', 
                            'owner_occupied_Qrange', 'lower_quartile_rent', 'median_rent', 'upper_quartile_rent', 'rent_Qrange', 'median_rent_utilities', 'num_vacant', 
                            'pct_large_family', 'pct_large_hh', 'ppl_per_owner_occupied_hh', 'ppl_per_rental_hh', 'num_born_to_unmarried', 'num_immigrants'], axis=1)

In [ ]:
column_list = cc_df.columns

corr_matrix = cc_df[column_list].corr()

plt.figure(figsize=(150, 150))
sns.heatmap(corr_matrix, square=True, annot=True, linewidths=0.5, cmap="YlGnBu")
plt.show()

I'll now eliminate an additional 30 variables. Urbanness and public transit use are highly correlated with population. Statistics on percentage of population that is young can be condensed into a single variable. Social security income correlates with population over 65. Different statistics around income are highly correlated to per capita income.  Statistics around children in two parent households can be condensed into a single variable. Low levels of education are highly correlated to poverty. Divorce statistics can be condensed into one variable, as can percents of mothers who work. Immigrant statistics can be condensed, and statistics about English speaking can be eliminated because they correlate with immigrant statistics. Length of stay in the state and city can be eliminated in favor of length of stay in the same home.

In [ ]:
cc_df = cc_df.drop(columns=['num_urban', 'age_pct_12to21', 'age_pct_16to24', 'pct_w_socsec_inc', 'pct_w_invest_inc', 'median_fam_income', 'median_hh_income', 'white_pci', 
                            'pct_two_parent_hh', 'pct_kids_under4_twophh', 'pct_kids_12_17_twophh', 'education_pct_under9', 'education_pct_not_HS', 'pct_unemployed', 'pct_female_divorced', 
                            'pct_male_divorced', 'pct_moms_under18_work', 'pctimm_3year_immigrants', 'pctimm_5year_immigrants', 'pctimm_8year_immigrants', 'pct_3year_immigrants', 
                            'pct_5year_immigrants', 'pct_8year_immigrants', 'pct_English_only', 'pct_English_notwell', 'pct_units_under3bed', 'pct_from_same_state', 'pct_5year_city_residents', 
                            'pct_5year_state_residents', 'pct_public_transit'], axis=1)

Now I'll search for pairs of variables with correlations over 0.6. 

In [ ]:
cols_minus_target = cc_df.drop(columns=['violent_crimes_per_100K'], axis=1)

column_list = cols_minus_target.columns

corr_matrix = cc_df[column_list].corr()

plt.figure(figsize=(150, 150))
sns.heatmap(corr_matrix, square=True, annot=True, linewidths=0.5, cmap="YlGnBu")
plt.show()

In [ ]:
sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                 .stack()
                 .sort_values(ascending=False))

corrs = pd.DataFrame(sol)

high_correlates = corrs.loc[(corrs[0] >= 0.6) | (corrs[0] <= -0.6 )] 

high_correlates

,,0
population,num_in_poverty,0.988448
pct_ppl_owner_occupied,pct_hh_owner_occupied,0.982186
pct_10year_immigrants,pct_foreign_born,0.959115
education_pct_collegeup,pct_management,0.951305
ppl_per_family,ppl_per_hh,0.937101
household_size,ppl_per_hh,0.878841
race_pct_hispanic,pct_ppl_dense_hh,0.864338
pct_w_wages,pct_employed,0.860659
household_size,ppl_per_family,0.834311
per_capita_income,pct_management,0.816737


In [ ]:
columns_to_eliminate = []

for i, j in high_correlates.index:
  first_corr = abs(cc_df[i].corr(cc_df['violent_crimes_per_100K']))
  second_corr = abs(cc_df[j].corr(cc_df['violent_crimes_per_100K']))
  if first_corr > second_corr:
    columns_to_eliminate.append(j)
  if second_corr > first_corr:
    columns_to_eliminate.append(i)

column_list = list(set(columns_to_eliminate))

column_list

['pct_ppl_owner_occupied',
 'race_pct_asian',
 'ppl_per_hh',
 'pct_hh_owner_occupied',
 'pct_w_public_asst',
 'race_pct_hispanic',
 'pct_w_wages',
 'pct_in_poverty',
 'race_pct_black',
 'per_capita_income',
 'household_size',
 'pct_foreign_born',
 'education_pct_collegeup',
 'pct_10year_immigrants',
 'pct_manufacturing',
 'pct_male_unmarried',
 'pct_employed',
 'pct_kids_two_parent_hh',
 'age_pct_65up',
 'hispanic_pci',
 'age_pct_12to29',
 'pct_manufacturing_2',
 'race_pct_white',
 'pct_pop_divorced',
 'pct_hh_no_phone',
 'median_bedrooms',
 'ppl_per_family',
 'population']

In [ ]:
cc_df = cc_df.drop(columns=column_list, axis=1)

cc_df

,pct_urban,pct_w_self_emp,pct_w_retirement_inc,black_pci,native_pci,asian_pci,other_pci,num_in_poverty,pct_prof_services,pct_management,pct_moms_under6_work,pct_born_to_unmarried,pctimm_10year_immigrants,pct_ppl_dense_hh,pct_housing_occupied,pct_vacant_boarded_up,pct_vacant_over6months,median_year_built,pct_hh_no_plumbing,rent_as_pct_hh_income,mortgage_as_pct_hh_income,non_mortgage_owner_as_pct_hh_income,pct_5year_home_residents,land_area,population_density,pct_police_drug_unit,violent_crimes_per_100K
0,100.00,1.55,18.39,13600,5725,27101,5115.0,227,28.82,50.73,44.56,0.36,30.93,0.39,98.37,3.12,37.50,1959,0.28,23.8,21.1,14.0,65.29,6.5,1845.9,0.0,41.02
1,100.00,1.11,22.85,18137,0,20074,5250.0,885,29.28,37.64,51.14,0.24,22.50,1.01,97.15,0.00,18.33,1958,0.14,27.6,20.7,12.5,71.27,10.6,2186.7,0.0,127.56
2,100.00,1.15,14.56,16644,21606,15528,5954.0,1389,21.52,32.48,66.08,0.88,35.63,2.03,95.68,0.92,7.54,1976,0.12,24.1,21.7,11.6,36.60,10.6,2780.9,0.0,218.59
3,0.00,0.24,18.33,9984,4941,3541,2451.0,2831,27.43,22.71,59.59,3.84,15.34,1.21,91.19,2.54,57.85,1939,0.87,28.7,20.6,14.5,56.70,5.2,3217.7,0.0,306.64
5,100.00,1.00,14.09,7382,10264,10753,7192.0,23223,26.83,23.42,62.96,1.58,45.48,2.11,91.81,2.09,26.22,1966,0.31,26.4,17.3,11.7,42.29,70.4,1995.7,0.0,442.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,100.00,1.12,14.41,8344,8590,3399,6470.0,13804,27.18,25.02,47.24,4.49,60.31,16.89,96.40,1.46,13.18,1973,0.55,29.8,22.6,11.7,39.19,16.7,3365.4,0.0,545.75
2211,100.00,0.47,18.23,5768,10910,1718,11471.0,2364,37.06,28.73,64.20,2.98,75.64,3.99,89.72,4.59,46.08,1966,0.12,23.8,17.3,14.4,49.12,7.3,1682.8,0.0,124.10
2212,100.00,0.51,23.56,13630,13197,17313,8532.0,2460,23.36,23.54,55.73,1.60,26.72,4.39,93.30,0.84,29.47,1967,0.27,30.5,23.9,13.1,46.73,27.5,1195.2,0.0,353.83
2213,0.00,1.70,10.29,6437,8271,171,4436.0,4021,25.16,20.87,50.57,2.35,20.71,9.64,85.39,5.61,67.21,1964,0.87,26.2,23.3,14.1,54.05,6.3,2142.2,0.0,691.17


The data has been whittled down from 102 variables to 26 variables.

In [ ]:
column_list = cc_df.columns

corr_matrix = cc_df[column_list].corr()

plt.figure(figsize=(150, 150))
sns.heatmap(corr_matrix, square=True, annot=True, linewidths=0.5, cmap="YlGnBu")
plt.show()

###Outliers

I'll create box plots and histograms to visually inspect for outliers and variable distribution.

In [ ]:
plt.figure(figsize=(25, 25))

for i in range(1, len(cc_df.columns)):
    plt.subplot(5, 6, i)
    plt.boxplot(cc_df[cc_df.columns[i - 1]])
    plt.title(cc_df.columns[i - 1])

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(25, 25))

for i in range(1, len(cc_df.columns)):
    plt.subplot(5, 6, i)
    plt.hist(cc_df[cc_df.columns[i - 1]])
    plt.title(cc_df.columns[i - 1])

plt.tight_layout()
plt.show()

I'll also look at outliers as a percentage of each variable.

In [ ]:
Y = cc_df['violent_crimes_per_100K']
X = cc_df.drop('violent_crimes_per_100K', axis=1)

for column_name in X.columns:
  print('\033[1m' + '{}'.format(column_name) + '\033[0m')
  q75, q25 = np.percentile(X[column_name], [75, 25])
  iqr = q75 - q25

  for threshold in np.arange(1,5,1):
      min_val = q25 - (iqr*threshold)
      max_val = q75 + (iqr*threshold)
      print("The threshold is: {}".format(threshold))
      print("Percentage of outliers is: {}".format((
          len((np.where((X[column_name] > max_val) 
                        | (X[column_name] < min_val))[0]))/1993) * 100
      ) + '\n')

The only variable with no outliers is pct_urban.

Variables that have outliers making up less than 10% of the data, I'll winsorize:


*   pct_w_retirement_inc
*   black_pci
*   pct_management
*   pct_moms_under6_work
*   pctimm_10year_immigrants
*   pct_vacant_over6months
*   median_year_built
*   rent_as_pct_hh_income
*   mortgage_as_pct_hh_income
*   non_mortgage_owner_as_pct_hh_income
*   pct_5year_home_residents













In [ ]:
col_list = ['pct_w_retirement_inc', 'black_pci', 'pct_management', 'pct_moms_under6_work', 'pctimm_10year_immigrants', 'pct_vacant_over6months', 'median_year_built', 
            'rent_as_pct_hh_income', 'mortgage_as_pct_hh_income', 'non_mortgage_owner_as_pct_hh_income', 'pct_5year_home_residents']

for column_name in col_list:
  X[column_name] = winsorize(X[column_name], (0.10, 0.10))

This leaves 13 variables with outliers.

In [ ]:
plt.figure(figsize=(25, 25))

for i in range(1, len(X.columns) + 1):
    plt.subplot(5, 6, i)
    plt.boxplot(X[X.columns[i - 1]])
    plt.title(X.columns[i - 1])

plt.tight_layout()
plt.show()

Variables with long tails might be best transformed using log:


*   pct_w_self_emp
*   native_pci
*   asian_pci
*   other_pci
*   num_in_poverty
*   pct_prof_services
*   pct_ppl_dense_hh
*   pct_vacant_boarded_up
*   pct_hh_no_plumbing
*   land_area
*   population_density
*   pct_police_drug_unit

In [ ]:
col_list_log = ['pct_w_self_emp', 'native_pci', 'asian_pci', 'other_pci', 'num_in_poverty', 'pct_prof_services', 'pct_ppl_dense_hh', 'pct_vacant_boarded_up', 'pct_hh_no_plumbing', 
                'land_area', 'population_density', 'pct_police_drug_unit']

for col_name in col_list_log:
  print('\033[1m' + '{}'.format(col_name) + '\033[0m')
  q75, q25 = np.percentile(X[col_name], [75, 25])
  iqr = q75 - q25

  for threshold in np.arange(1,5,1):
      min_val = q25 - (iqr*threshold)
      max_val = q75 + (iqr*threshold)
      print("The score threshold is: {}".format(threshold))
      print("Number of outliers is: {}".format(
          len((np.where((np.log(X[col_name]) > max_val) 
                        | (np.log(X[col_name]) < min_val))[0]))
      ) + '\n')

Transforming the data logarithmically worked for the following variables:

*   num_in_poverty
*   pct_ppl_dense_hh
*   land_area
*   population_density


I'll now add the logarithmically transformed columns to the dataframe X.

In [ ]:
X['log_num_in_poverty'] = np.log(X['num_in_poverty'])
X['log_pct_ppl_dense_hh'] = np.log(X['pct_ppl_dense_hh'])
X['log_land_area'] = np.log(X['land_area'])
X['log_population_density'] = np.log(X['population_density'])

Here's the breakdown of how the variables have been transformed so far:

None:
*   pct_urban



Winsorized:
*   pct_w_retirement_inc
*   black_pci
*   pct_management
*   pct_moms_under6_work
*   pctimm_10year_immigrants
*   pct_vacant_over6months
*   median_year_built
*   rent_as_pct_hh_income
*   mortgage_as_pct_hh_income
*   non_mortgage_owner_as_pct_hh_income
*   pct_5year_home_residents

Log Transformed:
*   num_in_poverty
*   pct_ppl_dense_hh
*   land_area
*   population_density

That leaves the following with outliers.  I will use Yeo-Johnson transformation on these:
*   pct_w_self_emp
*   native_pci
*   asian_pci
*   other_pci
*   pct_prof_services
*   pct_housing_occupied
*   pct_vacant_boarded_up
*   pct_hh_no_plumbing
*   pct_police_drug_unit







In [ ]:
col_list_yj = ['pct_w_self_emp', 'native_pci', 'asian_pci', 'other_pci', 'pct_prof_services', 'pct_housing_occupied', 'pct_vacant_boarded_up', 'pct_hh_no_plumbing', 'pct_police_drug_unit']

for column_name in col_list_yj:
  yeojohnson_version,_ = yeojohnson(X[column_name])
  jb_stats = jarque_bera(yeojohnson_version)
  norm_stats = normaltest(yeojohnson_version)
  print('\033[1m' + 'For variable {}: \n'.format(column_name) + '\033[0m' + str(jb_stats) + '\n' + str(norm_stats))

Yeo Johnson transformation works on just one of the variables, pct_w_self_emp.  Because Yeo Johnson may make interpreting the results of a linear regression model confusing and may alter the predictions made by the model, I will not plan to use Yeo-Johnson transformed pct_w_self_emp.

Though some of the variables still have outliers, I will plan to use the variables as they are currently transformed in the models I try.  Transforming them further might make the results difficult to interpret.

In [ ]:
plt.figure(figsize=(25, 25))

for i in range(1, len(X.columns) + 1):
    plt.subplot(5, 6, i)
    plt.boxplot(X[X.columns[i - 1]])
    plt.title(X.columns[i - 1])

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(25, 25))

for i in range(1, len(X.columns) + 1):
    plt.subplot(5, 6, i)
    plt.hist(X[X.columns[i - 1]])
    plt.title(X.columns[i - 1])

plt.tight_layout()
plt.show()

NameError: ignored

##**------------Selecting a Model-------------------------**

Preliminarily, I will run the data through six types of models.  I will then tune the parameters of whichever model yields the best initial result.


*   Decision Tree
*   Random Forest
*   K-nearest neighbors (KNN)
*   Support vector machines (SVM)
*   Gradient boosting with regression trees
*   Linear regression




In the tradeoff between interpretability and predictive power, I'm most interested in interpretability.  A model like a decision tree or linear regression might yield useful information about correlates to violent crime rates in a community; this could guide decisions for the community.  However, if those models have much lower success metrics than models that are more difficult to interpret but have high predictive power, I will select a model with better predictive power.

###Metrics to Evaluate Success

First, I will set up a function that displays success metrics.

In [ ]:
def metrics(test_data, predictions):
  print('Mean Absolute Error: ', mean_absolute_error(test_data, predictions))
  print('Mean Squared Error: ', mean_squared_error(test_data, predictions))
  print('Root MSE: ', rmse(test_data, predictions))
  print('Mean Absolute Percentage Error', np.mean(np.abs((test_data - predictions) / test_data)) * 100)

Now I'll run the data through a dummy regressor to see how well an 'empty' model performs.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=35)

In [ ]:
dummy_mean = DummyRegressor(strategy='mean')

dummy_mean.fit(X_train, y_train)

predictions = dummy_mean.predict(X_test)

dummy_score = dummy_mean.score(X_test, y_test)
print('Dummy Score: ', dummy_score)
metrics(y_test, predictions)

In [ ]:
dummy_median = DummyRegressor(strategy='median')

dummy_median.fit(X_train, y_train)

predictions = dummy_median.predict(X_test)

dummy_score = dummy_median.score(X_test, y_test)
print('Dummy Score: ', dummy_score)
metrics(y_test, predictions)

So the R^2 values to beat are -0.005 and -0.162

###Model 1: Decision Tree

In [ ]:
decision_tree = tree.DecisionTreeRegressor(
    criterion='mse',
    max_depth = 5
)

decision_tree.fit(X_train, y_train)

In [ ]:
dot_data = tree.export_graphviz(
    decision_tree, out_file=None, feature_names = X.columns, filled=True
)

graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
predictions = decision_tree.predict(X_test)

print('R^2: ', decision_tree.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(decision_tree, X, Y, cv=5)

This decision tree already performs better than the dummy regressors despite having wide-ranging cross-validation scores.

###Model 2: Random Forest

In [ ]:
rfr = ensemble.RandomForestRegressor(max_depth=5)

rfr.fit(X_train, y_train)

In [ ]:
predictions = rfr.predict(X_test)

print('R^2: ', rfr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
scores = cross_val_score(rfr, X, Y, cv=5)
scores

The random forest model performs yet better than the decision tree, with R^2 values hovering around 0.6.

###Model 3: KNN

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors = 10)

knn.fit(X_train, y_train)

In [ ]:
predictions = knn.predict(X_test)

print('R^2: ', knn.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(knn, X, Y, cv=5)

The KNN model is not nearly as successful as the random forest, with R^2 values hovering around 0.3.

###Model 4: SVM

In [ ]:
svr = SVR()

svr.fit(X_train, y_train)

In [ ]:
predictions = svr.predict(X_test)

print('R^2: ', svr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(svr, X, Y, cv=5)

The SVM model with the radial basis function as kernel performs even worse than one of the dummy regressors.  I will try two other kernels to see if these produce better results.

In [ ]:
svr = SVR(kernel='poly')

svr.fit(X_train, y_train)

predictions = svr.predict(X_test)

print('R^2: ', svr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
svr = SVR(kernel='sigmoid')

svr.fit(X_train, y_train)

predictions = svr.predict(X_test)

print('R^2: ', svr.score(X_test, y_test))
metrics(y_test, predictions)

The results of Support Vector Machines are still worse than the dummy models.

###Model 5: Gradient boosting with regression trees

In [ ]:
params = {
    'n_estimators' : 500,
    'loss' : 'deviance'
}

grd = ensemble.GradientBoostingRegressor()

grd.fit(X_train, y_train)

In [ ]:
predictions = grd.predict(X_test)

print('R^2: ', grd.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(grd, X, Y, cv=5)

The cross-validation scores for the gradient boosting model indicate possibly more success than the Random Forest model.  R^2 hovers around 0.6.

###Model 6: Linear Regression

Data fed into a linear regression model should obey the Gauss-Markov assumptions.

*   Linearity of the model in its coefficients (model will do this)
*   Error term should be zero on average*
*   Homoscedasticity*
*   Variables should not be highly collinear (fixed by feature engineering)
*   Error terms should be uncorrelated*
*   Features should be uncorrelated with errors*
*   Error terms should be normally distributed if we want to measure statistical significance of estimated coefficients*



*Some of these assumptions can be tested after the model is run.

In [ ]:
lrm = linear_model.LinearRegression()

lrm.fit(X_train, y_train)

In [ ]:
predictions = lrm.predict(X_test)

print('R^2: ', lrm.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(lrm, X, Y, cv=5)

The linear regression model appears to have similar predictive power to both the SVM and Random Forest models, with the model accounting for ~0.6 of the variation in the target.

Now I'll check some of the Gauss-Markov assumptions from above.

In [ ]:
#average error

errors = y_test - predictions
np.mean(errors)

In [ ]:
#distribution of errors / homoscedasticity

plt.scatter(predictions, errors)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.show()

In [ ]:
bart_stats = bartlett(predictions, errors)
lev_stats = levene(predictions, errors)

print(bart_stats[0], bart_stats[1])
print(lev_stats[0], lev_stats[1])

The results of the Bartlett and Levene tests, as well as visual inspection, show that the errors are heteroscedastic.  The average error term is not zero or close to it.

In [ ]:
#autocorrelation among errors
acf_data = acf(errors)

plt.plot(acf_data[1:])
plt.show()

From visual inspection, there doesn't seem to be strong correlation among error terms.

In [ ]:
#looking for normal distribution of error terms

rand_nums = np.random.normal(np.mean(errors), np.std(errors), len(errors))

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.scatter(np.sort(rand_nums), np.sort(errors))
plt.xlabel("the normally distributed random variable")
plt.ylabel("errors of the model")
plt.title("QQ plot")

plt.subplot(1,2,2)
plt.hist(errors)
plt.xlabel("errors")
plt.title("Histogram of the errors")

plt.tight_layout()
plt.show()

In [ ]:
jb_stats = jarque_bera(errors)
norm_stats = normaltest(errors)

print("Jarque-Bera test statistics is {0} and p value is {1}".format(jb_stats[0], jb_stats[1]))
print("Normality test statistics is {0} and p value is {1}".format(norm_stats[0], norm_stats[1]))

In [ ]:
errors.describe()

The low p-value results of the Jarque-Bera and Normality tests indicate that the errors are not normally distributed; however, visual inspection shows the error distribution is almost normal, and the Jarque-Bera and Normality tests are known to be highly sensitive.

In [ ]:
X_train = sm.add_constant(X_train)

results = sm.OLS(y_train, X_train).fit()

results.summary()

The variables that have statistical significance include: pct_urban, asian_pci, other_pci, pct_born_to_unmarried, pct_housing_occupied, pct_vacant_boarded_up, median_year_built, non_mortgage_owner_as_pct_hh_income, pct_5year_home_residents, pct_police_drug_unit, log_num_in_poverty, log_pct_ppl_dense_hh.

Translating some of this into a sentence, it seems that urbanness, having more residents in dense housing, more poverty, and more police in drug units are positively correlated with violent crime rates.  Having more long-term home residents, homeowners who own their homes, newer homes, and less vacant housing is negatively correlated with violent crime rates.  As indicated in the correlation matrices above, the biggest correlate with violent crime rates is percent of children born to unmarried parents.

The fact that the one police statistic we have is statistically significant makes me wish that I had access to complete police data.  It's likely some important variables are not included in the dataset.

###Three Best Models

The three models with the best success metrics after an initial run are random forest, gradient boosting with regression trees, and linear regression, all of which have R^2 values hovering around 0.6.  I will tune each of these models to pick which model is the best balance of interpretability and predictive power.

##**------------Tuning the Models-------------------------**

###Tuning Random Forest

I will use GridSearchCV to find the set of parameters to use with the random forest model to maximize success metrics.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=35)

In [ ]:
rfr = ensemble.RandomForestRegressor()

params = {
    'n_estimators' : [100, 250, 500, 750, 1000], 
    'max_depth' : [5, 10, 15, 20]
}

model = GridSearchCV(rfr, params, cv=5)
model.fit(X_train, y_train)
model.best_params_

In [ ]:
rfr = ensemble.RandomForestRegressor(n_estimators=250, max_depth=20)

rfr.fit(X_train, y_train)

predictions = rfr.predict(X_test)

print('R^2: ', rfr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
rfr = ensemble.RandomForestRegressor(n_estimators=250, max_depth=None)

rfr.fit(X_train, y_train)

predictions = rfr.predict(X_test)

print('R^2: ', rfr.score(X_test, y_test))
metrics(y_test, predictions)

A max_depth of None is only marginally better, and marginally slower, than max_depth of 20.  

In [ ]:
feature_importance = rfr.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(18, 12))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center', color='orange')
plt.yticks(pos, np.array(X.keys())[sorted_idx])
plt.title('Feature Importance (MDI)')

fig.tight_layout()
plt.show()

###Tuning Gradient Boosting

In [ ]:
params = {
    'n_estimators' : [100, 250, 500, 750, 1000], 
    'learning_rate' : [0.05, 0.1, 0.2, 0.3], 
    'max_depth' : [3, 5, 7]
}

grd = ensemble.GradientBoostingRegressor()

model = GridSearchCV(grd, params, cv=5)
model.fit(X_train, y_train)
model.best_params_

In [ ]:
grd = ensemble.GradientBoostingRegressor(n_estimators=250, max_depth=3, learning_rate = 0.05)

grd.fit(X_train, y_train)

predictions = grd.predict(X_test)

print('R^2: ', grd.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
feature_importance = grd.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(18, 12))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center', color='red')
plt.yticks(pos, np.array(X.columns)[sorted_idx])
plt.title('Feature Importance (MDI)')

fig.tight_layout()
plt.show()

###Linear Regression with Ridge, Lasso, and ElasticNet

I'll try ridge, lasso, and elasticnet models and will tune them to find the optimal linear regression model.

####Ridge

In [ ]:
alphas=[10**10, 10**20, 10**30, 10**40, 10**50]

ridge_crossval = RidgeCV(alphas=alphas, cv=5)
best_crossval = ridge_crossval.fit(X_train, y_train)

best_crossval.alpha_

In [ ]:
ridgeregr = Ridge(alpha=10**10) 
ridgeregr.fit(X_train, y_train)

predictions = ridgeregr.predict(X_test)

print('R^2: ', ridgeregr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(ridgeregr, X, Y, cv=5)

####Lasso

In [ ]:
alphas=[10**10, 10**20, 10**30, 10**40, 10**50]

lasso_crossval = LassoCV(alphas=alphas)
best_crossval = lasso_crossval.fit(X_train, y_train)

alpha = best_crossval.alpha_

scientific_notation = '{:e}'.format(alpha)
print(scientific_notation)

In [ ]:
lassoregr = Lasso(alpha = 10**50)
lassoregr.fit(X_train, y_train)

predictions = lassoregr.predict(X_test)

print('R^2: ', lassoregr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(lassoregr, X, Y, cv=5)

####ElasticNet

In [ ]:
alphas=[10**10, 10**20, 10**30, 10**40, 10**50]

elasticnet_crossval = ElasticNetCV(alphas=alphas)
best_crossval = elasticnet_crossval.fit(X_train, y_train)

alpha = best_crossval.alpha_
l1_ratio = best_crossval.l1_ratio_

scientific_notation = '{:e}'.format(alpha)
print(scientific_notation)
print(l1_ratio)

In [ ]:
elasticregr = ElasticNet(alpha = 10**50, l1_ratio=0.5)
elasticregr.fit(X_train, y_train)

predictions = elasticregr.predict(X_test)

print('R^2: ', elasticregr.score(X_test, y_test))
metrics(y_test, predictions)

In [ ]:
cross_val_score(elasticregr, X, Y, cv=5)

None of these three methods of linear regression works well at all.  This could be because the target seems to be a function, largely, of one variable in particular, which has a large coefficient as a result.  These three methods penalize large coefficients, which may be inescapable in this data set.  For linear regression, the most effective is simple ordinary least squares.

##**------------Comparing the Models-------------------------**

I will now compare the success metrics for the three best-performing models.

In [ ]:
comparison = pd.DataFrame()

comparison['Model'] = ['Best Random Forest', 'Best Gradient Boosting', 'Ordinary Least Squares']
comparison['R^2'] = [0.6036, 0.6319, 0.6189]
comparison['Mean Absolute Error'] = [262.50, 250.68, 264.04]
comparison['Mean Squared Error'] = [158632, 147107, 152279]
comparison['Root Mean Squared Error'] = [398.29, 383.55, 390.23]
comparison['Mean Absolute Percentage Error'] = [93.83, 81.55, 91.30]

comparison

In [ ]:
C = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255]])

plt.figure(figsize=(18, 18))

plt.subplot(2, 3, 1)
plt.bar(comparison['Model'], comparison['R^2'], color=C/255.0)
plt.title('R^2')
plt.ylim((0.4, 1))
plt.xticks(rotation='vertical', fontsize=14)

plt.subplot(2, 3, 2)
plt.bar(comparison['Model'], comparison['Mean Absolute Error'], color=C/255.0)
plt.title('Mean Absolute Error')
plt.ylim((200, 300))
plt.xticks(rotation='vertical', )

plt.subplot(2, 3, 3)
plt.bar(comparison['Model'], comparison['Mean Squared Error'], color=C/255.0)
plt.title('MSE')
plt.ylim((140000, 170000))
plt.xticks(rotation='vertical')

plt.subplot(2, 3, 4)
plt.bar(comparison['Model'], comparison['Root Mean Squared Error'], color=C/255.0)
plt.title('RMSE')
plt.ylim((380, 410))
plt.xticks(rotation='vertical')

plt.subplot(2, 3, 5)
plt.bar(comparison['Model'], comparison['Mean Absolute Percentage Error'], color=C/255.0)
plt.title('MAPE')
plt.ylim((75, 100))
plt.xticks(rotation='vertical')

red_patch = mpatches.Patch(color='red', label='Random Forest')
green_patch = mpatches.Patch(color='green', label='Gradient Boosting')
blue_patch = mpatches.Patch(color='blue', label='OLS')
plt.legend(handles=[red_patch, green_patch, blue_patch])

plt.tight_layout()
plt.show()

###Choosing a Model

Of these models, **gradient boosting with regression trees** has the lowest errors and by a narrow margin the highest R^2 value.  For predictive power, I would choose this model.

However, I stated at the beginning at the exploration that my primary goal is interpretability.  Therefore, I would choose **ordinary least squares**, because looking at the coefficients gives me an even more explicit picture of what's going on than the list of feature importances provided by the gradient boosting model.

##**------------Disclaimers and Analysis-------------------------**

I note again here that the data does not fit all of the Gauss-Markov assumptions, and therefore, the statistical significance of each coefficient of the linear regression may be questioned.  In addition, the errors of the OLS model seem to increase for higher numbers, which is something to keep in mind if the OLS model were used for prediction.  This could be addressed with its own ensemble / boosting strategy, and it may be related to the outliers still present in the data.

Importantly, the data included in the dataframe X only accounts for ~60% of the variance in violent crime rates, per R^2.  There must be variables not measured by the LEMAS, FBI, and census data that are important for understanding rates of violent crime.  Complete police data might provide one set of clues.  Measures of infrastructure, services, and employment opportunities might be valuable.  Finding additional data would be best advised by someone with area knowledge.

If interpretive power were not a goal, principal component analysis could be used on the original 102-variable dataframe for feature engineering.  This might result in higher accuracy, but it does not allow for easy interpretation.

Here I will re-print the metrics generated by statsmodels' OLS and use the statistical significance of the coefficients to write an incredibly rough, unscientific formula for violent crime per 100,000 residents as a function of community attributes.

In [ ]:
X_train = sm.add_constant(X_train)

results = sm.OLS(y_train, X_train).fit()

results.summary()

In [ ]:
print('Mean violent crimes per 100K: ', np.mean(cc_df['violent_crimes_per_100K']))
print('Median violent crimes per 100K: ', np.median(cc_df['violent_crimes_per_100K']))
print('Maximum violent crimes per 100K: ', np.max(cc_df['violent_crimes_per_100K']))
print('Minimum violent crimes per 100K: ', np.min(cc_df['violent_crimes_per_100K']))

violent_crime_per_100K ~=  **114.32** x percent of children born to unmarried parents + **93.1** x the (log) percent of residents in dense housing + **77.1** x the (log) number of residents under the poverty line - **52.7** x the percent of household income homeowners with no mortgages spend on their homes + **16.6** x percent of housing boarded up - **11.7** x percent of housing occupied + **8.4** x percent of police who are in drug units + **4.1** x percent of residents who have been in the same home for over five years + **0.65** x urbanness + **0.004** x Asian per capita income + **0.003** x per capita income for other racial categories  + **4,345**

The data paints a very stereotypical portrait of troubled neighborhoods, the same portrait I was taught growing up in the 90's; some of the variables are highly correlated with racial demographics. Fortunately, there are contemporary conversations questioning why organizations like the FBI and LEMAS ask the questions they do, and what the underlying causes are that result in these correlations.  There are also conversations about the role policing has in a community and the feedback loop that results when over-policed communities have more arrests, resulting in even more policing.

The most interesting result, for me, is that poverty and its direct correlates are not the highest predictors of violent crime.  They're part of a constellation of related factors.  One funny and questionable part of this data, for me, is that the surveyors thought to count unmarried mothers but not unmarried fathers; it's like the data collection process itself is accusing unmarried mothers of ruining the neighborhood.  I changed that variable to 'unmarried parents.'